In [1]:
#导入库与数据集
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import neighbors
from tqdm import tqdm

raw=pd.read_csv('exp3.csv')
label=pd.read_csv('label.csv')
label.head()

,index,gameDuration
0,20000,1671
1,20001,1227
2,20002,1458
3,20003,1317
4,20004,2091


In [2]:
def geten(df):
    en=pd.DataFrame()
    
    raw1=df.iloc[:,12:43]
    raw2=df.iloc[:,44:78]
    raw1['t1_kills']=raw1['player1_kills']+raw1['player2_kills']+raw1['player3_kills']+raw1['player4_kills']+raw1['player5_kills']
    raw1['t1_deaths']=raw1['player1_deaths']+raw1['player2_deaths']+raw1['player3_deaths']+raw1['player4_deaths']+raw1['player5_deaths']
    raw1['t1_assists']=raw1['player1_assists']+raw1['player2_assists']+raw1['player3_assists']+raw1['player4_assists']+raw1['player5_assists']
    raw2['t2_kills']=raw2['player6_kills']+raw2['player7_kills']+raw2['player8_kills']+raw2['player9_kills']+raw2['player10_kills']
    raw2['t2_deaths']=raw2['player6_deaths']+raw2['player7_deaths']+raw2['player8_deaths']+raw2['player9_deaths']+raw2['player10_deaths']
    raw2['t2_assists']=raw2['player6_assists']+raw2['player7_assists']+raw2['player8_assists']+raw2['player9_assists']+raw2['player10_assists']
    en['kills'] = raw1['t1_kills']+raw2['t2_kills']
    en['assists'] = raw1['t1_assists']+raw2['t2_assists']
    en['deaths'] = raw1['t1_deaths']+raw2['t2_deaths']
    
    raw1=df.loc[:,['player1_goldEarned','player2_goldEarned','player3_goldEarned','player4_goldEarned','player5_goldEarned']]
    raw2=df.loc[:,['player6_goldEarned','player7_goldEarned','player8_goldEarned','player9_goldEarned','player10_goldEarned']]
    raw1['gold1'] = raw1.apply(lambda x: x.sum(), axis=1)
    raw2['gold2'] = raw2.apply(lambda x: x.sum(), axis=1)
    en['gold'] = raw1['gold1']+raw2['gold2']
    
    #en['gameDuration'] = df['gameDuration']                                                                                                                        
    return en         

In [3]:
#初始化DataFrame文件
df=raw.loc[20000:]
df=df.reset_index(drop=True)
df=pd.concat([df,label],axis=1)
df=df.reindex(np.random.permutation(len(df)))
df.head()

,index,queueId,mapId,seasonId,team1_win,team1_firstBlood,team1_firstTower,team1_firstInhibitor,team1_firstBaron,team1_firstDragon,...,player9_lane,player10_championId,player10_kills,player10_deaths,player10_assists,player10_goldEarned,player10_role,player10_lane,index,gameDuration
1184,21184,420,11,13,1,False,True,True,True,True,...,BOTTOM,117,1,4,5,5319,DUO_SUPPORT,BOTTOM,21184,1468
55680,75680,430,11,13,1,False,False,False,False,True,...,BOTTOM,555,2,4,1,5606,DUO_SUPPORT,BOTTOM,75680,1344
57137,77137,420,11,13,1,True,True,True,False,True,...,JUNGLE,412,0,11,8,4964,DUO_SUPPORT,BOTTOM,77137,1285
32098,52098,420,11,13,0,False,False,False,False,False,...,MIDDLE,120,9,5,12,13441,NONE,JUNGLE,52098,1674
7498,27498,420,11,13,1,True,True,False,False,True,...,NONE,112,1,5,0,6177,DUO_SUPPORT,NONE,27498,1145


In [4]:
#提取特征
train=geten(df)
train.head()

,kills,assists,deaths,gold
1184,35,70,35,87933
55680,39,40,39,78877
57137,50,69,50,80472
32098,75,100,75,114471
7498,26,43,26,63754


In [5]:
train=(train-train.min())/(train.max()-train.min())
train['gameDuration']=df['gameDuration']
target=df['gameDuration']
target=((target/35).astype('int')+1)*35
target.head()

1184     1470
55680    1365
57137    1295
32098    1680
7498     1155
Name: gameDuration, dtype: int32

In [6]:
x_train=train.iloc[0:48000,0:4]
x_test=train.iloc[48001:,0:4]
y_train=target.iloc[0:48000]
y_test=train.iloc[48001:,4]

x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values
print (x_train)
print (y_train)
print (x_test)
print (y_test)

[[0.19886364 0.17857143 0.19774011 0.33798223]
 [0.22159091 0.10204082 0.22033898 0.29967428]
 [0.28409091 0.17602041 0.28248588 0.30642132]
 ...
 [0.26704545 0.1505102  0.26553672 0.39983926]
 [0.35795455 0.24234694 0.3559322  0.43664975]
 [0.11363636 0.05357143 0.11299435 0.21274958]]
[1470 1365 1295 ... 1750 1785 1050]
[[0.4375     0.22959184 0.43502825 0.55420051]
 [0.17045455 0.10714286 0.16949153 0.32208968]
 [0.39772727 0.19132653 0.39548023 0.50102369]
 ...
 [0.61363636 0.32142857 0.61016949 0.57453469]
 [0.31818182 0.16326531 0.31638418 0.39493655]
 [0.56818182 0.29336735 0.56497175 0.61913283]]
[2249 1449 2061 ... 2081 1597 2289]


In [7]:
#KNN分类算法（分隔区间法）
min=float('inf')
for k in tqdm(range(5,70)):  #网格搜索最佳K值
    knn = neighbors.KNeighborsClassifier(k)
    knn.fit(x_train,y_train)
    pre=knn.predict(x_test)
    err=metrics.mean_squared_error(pre,y_test)
    if err<min:
        min=err
        best=k
print('best k is:',best,'min error is:',min)

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:40<00:00,  1.62it/s]

best k is: 63 min error is: 15444.495707975664


In [8]:
knn = neighbors.KNeighborsClassifier(best)
knn.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=63)

In [9]:
#如果预测值与真实值误差小于100那么认为预测是“合格”的，计算合格率
n=0
for i in range(len(pre)):
    if abs(pre[i]-y_test[i])<100:
        n=n+1
print('合格率为：',n/len(pre))

合格率为： 0.8434036169680806


In [10]:
#KNN回归
df=raw.loc[20000:]
df=df.reset_index(drop=True)
df=pd.concat([df,label],axis=1)
df=df.reindex(np.random.permutation(len(df)))

train=geten(df)
train=(train-train.min())/(train.max()-train.min())
train['gameDuration']=df['gameDuration']
target=df['gameDuration']

x_train=train.iloc[0:48000,0:4]
x_test=train.iloc[48001:,0:4]
y_train=target.iloc[0:48000]
y_test=train.iloc[48001:,4]
x_train=x_train.values
x_test=x_test.values
y_train=y_train.values
y_test=y_test.values

In [11]:
#网格搜索最佳K值
min=float('inf')
for k in tqdm(range(5,70)):
    knn = neighbors.KNeighborsRegressor(k)
    knn.fit(x_train,y_train)
    pre=knn.predict(x_test)
    err=metrics.mean_squared_error(pre,y_test)
    if err<min:
        min=err
        best=k
print('best k is:',best,'min error is:',min)

100%|██████████████████████████████████████████████████████████████████████████████████| 65/65 [00:20<00:00,  3.18it/s]

best k is: 25 min error is: 12326.612138744895


In [12]:
knn = neighbors.KNeighborsRegressor(best)
knn.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=25)

In [13]:
#如果预测值与真实值误差小于100那么认为预测是“合格”的，计算合格率
n=0
for i in range(len(pre)):
    if abs(pre[i]-y_test[i])<100:
        n=n+1
print(n/len(pre))

0.8784898741561796


In [20]:
#将预测结果写入csv文件中
df2=raw.loc[0:19999]
test=geten(df2)
test=(test-test.min())/(test.max()-test.min())
result=knn.predict(test)
test['gameDuration']=result
test=test[['gameDuration']]
test.to_csv("test.csv",index=True)